In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [11]:
from google.adk.agents import Agent, LlmAgent
from google.adk.tools import google_search
from google.adk.models.google_llm import Gemini
from google.genai import types
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    initial_delay=1,
    exp_base=7,
    http_status_codes=[429, 500, 503, 504]
)

In [9]:
personal_agent = Agent(
    name="Personal_Assistant",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="You are a personel assistant for users. Provide answers based on user queries",
    tools=[google_search]
)

In [13]:
memery_service = InMemoryMemoryService()
session_service = InMemorySessionService()

In [14]:
runner = Runner(
    app_name="PersonelApp",
    agent=personal_agent,
    session_service=session_service,
    memory_service=memery_service
)

App name mismatch detected. The runner is configured with app name "PersonelApp", but the root agent was loaded from "D:\Google Capstone\BiteMate\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [16]:
# Define constants used throughout the notebook
APP_NAME = "PersonelApp"
USER_ID = "demo_user"

In [29]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    return text


print("✅ Helper functions defined.")


✅ Helper functions defined.


In [23]:
response = await run_session(
    runner,
    "what is photosysthesis?",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > what is photosysthesis?


In [24]:
print(response)

Photosynthesis is a fundamental biological process through which plants, algae, and some types of bacteria convert light energy, typically from the sun, into chemical energy. This chemical energy is stored in sugar molecules, which the organisms then use for their metabolic processes and growth.

Here's a detailed look at the process:

*   **Inputs:** The primary raw materials for photosynthesis are carbon dioxide (CO2), absorbed from the atmosphere, and water (H2O), absorbed from the soil.
*   **Energy Input:** Light energy, most commonly sunlight, serves as the energy source to drive the chemical reactions.
*   **Key Molecule:** Chlorophyll, the green pigment found in plants, is essential for absorbing light energy.
*   **Outputs:** During photosynthesis, carbon dioxide and water are chemically converted into glucose (a sugar) and oxygen (O2). The oxygen is released into the atmosphere as a byproduct.

This process predominantly takes place in the chloroplasts within plant cells and 

In [30]:
response = await run_session(
    runner,
    "what did i asked",
    "conversation-01",  # Session ID
)

In [31]:
print(response)

You asked "what is photosynthesis?" and "what did I ask?".


In [32]:
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset